Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as any collaborators you worked with:

In [ ]:
COLLABORATORS = ""

## To receive credit for this assignment, you must also fill out the [AI Use survey](https://forms.gle/ZhR5k8TdAeN8rj4CA)


---

In [ ]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
import pandas as pd

# Final Project

This notebook will provide a brief structure and rubric for presenting your final project. 

The purpose of the project is 2-fold
* To give you an opportunity to work on a problem you are truly interested in (as this is the best way to actually learn something)
* To demonstrate to me that you understand the overall workflow of problem solving from problem selection to implementation to discussion 

You can choose any subject area that interests you as long as there is a computational component to it.  However, please do not reuse projects or homeworks you have done in other classes.  This should be **your** original work.

**You can work in teams, but clearly identify each persons contribution** and every team member should hand in their own copy of the notebook.

### Structure
There are 5 parts for a total of 100 points that provide the overall structure of a mini research project.

* Abstract
* Introduction and Problem Description
* Brief discussion of Computational approach and import of any additional packages
* Implementation including tests
* Discussion of results and future directions

For grading purposes, please try to make this notebook entirely self contained. 

The project is worth about 2 problem sets and should be of comparable length (please: I will have about 100 of these to read and I am not expecting full 10 page papers).  The actual project does not necessarily have to work but in that case you should demonstrate that you understand why it did not work and what steps you would take next to fix it.

Have fun

## Abstract [10 pts]

Provide a 1-2 paragraph abstract of the project in the style of a research paper.  The abstract should contain

* A brief description of the problem
* A brief justification describing why this problem is important/interesting to you
* A general description of the computational approach
* A brief summary of what you did and what you learned


YOUR ANSWER HERE
Every year, extreme weather events cause damage to homes across the United States, many of which are insured. For a property insurance carrier, it is important to model the risks and expected losses that arise from underwriting home insurance policies, especially in states that experience a high frequency of extreme weather events. Thus, this project aims to use FEMA data to model the claims, expected losses, and profits that an insurer might face in states like Florida and Washington. I am personally interested in exploring this problem because I am an aspiring actuary and would like to explore Catastrophe Modeling as applied to the property insurance space, which is full of complex problems and real-world implications.

I began by collecting data from FEMA's National Risk Index resources, which included fields like proprietary risk scores and Expected Annual Loss (EAL) on a county level. Taking the EAL and other assumptions about the insurance carrier, like its market share, base claim rate, and the average value of the homes it insures in each state, Generalized Linear Models (GLMs) are fitted to predict the severity and frequency parameters for distributions that are then used to model claims in a Monte Carlo simulation. In the claims simulation step, typical insurance policy mechanics like deductibles and coinsurance rates are modeled with 

## Introduction [15 pts]

In ~4-5 paragraphs, describe 
* The general problem you want to solve
* Why it is important and what you hope to achieve.

Please provide basic **references**, particularly if you are reproducing results from a paper. Also include any basic equations you plan to solve. 

Please use proper spelling and grammar. 

I am interested in exploring how an insurance company decides whether or not to underwrite policies (i.e. provide insurance) in areas or states that experience a high number of extreme-weather events. This task requires modeling the frequency and severity of potential claims that may occur\, in order to understand the potential costs an insurer may face in such an area.

Significance of the Problem: I am personally interested in exploring this problem because I am an aspiring actuary, which is a risk professional who typically works with insurance companies to forecast future outcomes and their financial costs. The problem itself is interesting because of the dynamic and costly nature of weather-related catastrophes, as climate change and new regulations continually affects the Property & Casualty insurance indsutry. For example, California passed new legislation this Summer which allows insurance companies to develop Wildfire Catastrophe Models to aid their fire insurance underwriting, whereas beforehand, most companies chose to stay out of California by default because of the high-level of risk that they were not allowed to properly estimate.   

Potential computational tools: Monte Carlo or other stochastic simulations can be useful in forecasting a set of potential outcomes in this problem by randomly generating factors like total claim costs in a given state, which are the primary expense for insurance companies, and therefore an important indicator of whether a state that experiences frequent weather catastrophes is a viable market. If sufficient data exists for training, other Machine Learning approaches like Generalized Linear Models (commonly used by actuaries) or XGBoost can be deployed to predict expected losses or claim probabilities.   

Overall, I hope to gain some experience in exploring complex problems in the insurance industry, especially in a practice as dense and dynamic as Catastrophe Modeling. I am excited to see how my project evolves from making naive assumptions in the model to hopefully growing in its faithfulness to real-world scenarios by incorporating more data-driven methods.

### References

YOUR ANSWER HERE

## Computational  Methods [10 pts]

Describe the specific approach you will take to solve some concrete aspect of the general problem. 

You should  include all the numerical or computational methods you intend to use.  These can include methods or packages  we did not discuss in class but provide some reference to the method. You do not need to explain in detail how the methods work, but you should describe their basic functionality and justify your choices. 




YOUR ANSWER HERE

**If you need to install or import any additional python packages,  please provide complete installation instructions in the code block below**


In [ ]:
# Provide complete installation or import information for external packages or modules here e.g.

#pip install somepackage
# from somepackage import blah

## Implementation [50 pts]

Use the Markdown and Code blocks below to implement and document your methods including figures.  Only the first markdown block will be a grading cell but please add (not copy) cells in this section to organize your work. 

Please make the description of your problem readable by interlacing clear explanatory text with code (again with proper grammar and spelling). 
All code should be well described and commented.

For at least one routine you code below, you should provide a test block (e.g. using `numpy.testing` routines, or a convergence plot) to validate your code.  

An **important** component of any computational paper is to demonstrate to yourself and others that your code is producing correct results.

YOUR ANSWER HERE

## Discussion [15 pts]

Evaluate the results of your project including 
* Why should I believe that your numerical results are correct (convergence, test cases etc)?
* Did the project work (in your opinion)?
* If yes:  what would be the next steps to try
* If no:  Explain why your approach did not work and what you might do to fix it.


YOUR ANSWER HERE